# Libraries

In [2]:
import  pandas            as pd
import  numpy             as np
import  plotly.express    as px
import  matplotlib.pyplot as plt
import  plotly.graph_objs as go
import  plotly.io         as pio
from    plotly.subplots         import make_subplots
from    sklearn.model_selection import train_test_split
from    sklearn.impute          import SimpleImputer
from    imblearn.over_sampling  import SMOTE
from    sklearn.preprocessing   import StandardScaler
from    sklearn.ensemble        import RandomForestClassifier
from    xgboost                 import XGBClassifier
from    sklearn.neural_network  import MLPClassifier
from    sklearn.ensemble        import AdaBoostClassifier
from    sklearn.model_selection import RandomizedSearchCV
from    sklearn.model_selection import cross_validate
from    sklearn.metrics         import make_scorer
from    sklearn.metrics         import accuracy_score
from    sklearn.metrics         import confusion_matrix
from    sklearn.metrics         import precision_score
from    sklearn.metrics         import recall_score
from    sklearn.metrics         import f1_score
from    sklearn.metrics         import roc_auc_score
from    sklearn.metrics         import ConfusionMatrixDisplay
from    sklearn.metrics         import RocCurveDisplay

In [3]:
RANDOM_SEED = np.random.seed(31)

In [4]:
cd ..\

c:\Users\victo\Downloads\00. Estudo Data Science\aps_failure_truck_prediction


c:\Users\victo\Downloads\00. Estudo Data Science\aps_failure_truck_prediction\venv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


# Train

In [5]:
df_train = pd.read_csv('data\\original\\air_system_previous_years.csv',
                       encoding='utf-8',
                       na_values=['na']
                      )
df_train.head()

,class,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,...,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
0,neg,76698,NaN,2.130706e+09,280.0,0.0,0.0,0.0,0.0,0.0,...,1240520.0,493384.0,721044.0,469792.0,339156.0,157956.0,73224.0,0.0,0.0,0.0
1,neg,33058,NaN,0.000000e+00,NaN,0.0,0.0,0.0,0.0,0.0,...,421400.0,178064.0,293306.0,245416.0,133654.0,81140.0,97576.0,1500.0,0.0,0.0
2,neg,41040,NaN,2.280000e+02,100.0,0.0,0.0,0.0,0.0,0.0,...,277378.0,159812.0,423992.0,409564.0,320746.0,158022.0,95128.0,514.0,0.0,0.0
3,neg,12,0.0,7.000000e+01,66.0,0.0,10.0,0.0,0.0,0.0,...,240.0,46.0,58.0,44.0,10.0,0.0,0.0,0.0,4.0,32.0
4,neg,60874,NaN,1.368000e+03,458.0,0.0,0.0,0.0,0.0,0.0,...,622012.0,229790.0,405298.0,347188.0,286954.0,311560.0,433954.0,1218.0,0.0,0.0


In [6]:
df_isna = df_train.drop(columns='class')
df_isna = df_isna.isna().sum().sort_values(ascending=False).to_frame('total_nan')
df_isna['total_data'] = df_train.shape[0]
df_isna['pct_nan'] = (df_isna['total_nan'] / df_isna['total_data'] * 100).round(2)
df_isna

,total_nan,total_data,pct_nan
br_000,49264,60000,82.11
bq_000,48722,60000,81.20
bp_000,47740,60000,79.57
bo_000,46333,60000,77.22
ab_000,46329,60000,77.22
...,...,...,...
ck_000,338,60000,0.56
ci_000,338,60000,0.56
cj_000,338,60000,0.56
bt_000,167,60000,0.28


In [7]:
fig = px.bar(df_isna,
             x=df_isna.index,
             y='pct_nan',
             text_auto='pct_nan'
             )
fig.update_layout(height=400,
                  width=800,
                  showlegend=True,
                  margin=dict(l=10, r=10, b=10, t=10),
                  yaxis=dict(title='Total NaN (%)', dtick=10),
                  xaxis=dict(title='Feature')
                  )
fig.show()

In [8]:
# Remove features with over 50% of NaN data
features_low_nan = df_isna[df_isna['pct_nan'] <= 50].index.to_list()

In [9]:
df_features = df_train[features_low_nan].astype(float)

# Change class object to int dummies
map_class = {'neg': 0, 'pos': 1}
df_target = df_train['class'].map(map_class).to_frame('class').astype(int)

In [10]:
df_count_class = df_target.value_counts().to_frame()
df_count_class

,count
class,
0,59000
1,1000


In [11]:
fig = px.bar(df_count_class,
             x=df_count_class.index.get_level_values('class'),
             y='count',
             text_auto='count'
             )
fig.update_layout(height=400,
                  width=800,
                  showlegend=True,
                  margin=dict(l=10, r=10, b=10, t=10),
                  yaxis=dict(title='Total'),
                  xaxis=dict(title='Class',
                             tickvals=['neg', 'pos'],
                             ticktext=['Negative', 'Positive'])
                  )
fig.show()

In [12]:
# Calculate events proportion
total     = df_target.shape[0]
non_event = df_target[df_target['class'] == 0].shape[0]
event     = df_target[df_target['class'] == 1].shape[0]

non_event_percent = round(non_event / total * 100, 2)
event_percent     = round(event / total * 100, 2)

df_percent_events = pd.DataFrame([non_event_percent, event_percent],
                                 columns=['percent'],
                                 index=['non_event', 'event']
                                 )
df_percent_events

,percent
non_event,98.33
event,1.67


In [13]:
fig = px.bar(df_percent_events,
             x=df_percent_events.index,
             y='percent',
             text_auto=True
             )
fig.update_layout(height=400,
                  width=800,
                  showlegend=True,
                  margin=dict(l=10, r=10, b=10, t=10),
                  yaxis = dict(title='Total (%)'),
                  xaxis = dict(title='Class',
                               tickvals=['non_event', 'event'],
                               ticktext=['Negative', 'Positive'])
                  )
fig.show()

In [14]:
# Identify features with zero variance
features_zero_variance = [col for col in df_features if df_features[col].std() == 0]
print(features_zero_variance)
df_features.drop(columns=features_zero_variance, inplace=True)

['cd_000']


## Split train and validation

In [15]:
X_train, X_val, y_train, y_val = train_test_split(df_features, df_target, test_size=0.3, random_state=RANDOM_SEED)

## Input data on NaN

In [16]:
# Use the median to replace NaN values
imputer = SimpleImputer(missing_values=np.nan, strategy='median')
imputer.fit(X_train)

# # Save the imputer to a file
# with open('train_artifacts\\simple_imputer.pkl', 'wb') as file:
#     pickle.dump(imputer, file)

# # Load the imputer from the file
# with open('train_artifacts\\simple_imputer.pkl', 'rb') as file:
#     imputer = pickle.load(file)

# Fill NaN on train
X_train_filled_nan = imputer.transform(X_train)
X_train_filled_nan = pd.DataFrame(X_train_filled_nan, columns=X_train.columns)
print(X_train_filled_nan.isna().sum().sum())

# Fill NaN on validation
X_val_filled_nan = imputer.transform(X_val)
X_val_filled_nan = pd.DataFrame(X_val_filled_nan, columns=X_train.columns)
print(X_val_filled_nan.isna().sum().sum())

0
0


In [17]:
df_val = X_val_filled_nan.join(y_val)
df_val.head()

,bl_000,bk_000,cg_000,ch_000,cf_000,co_000,ad_000,cx_000,cy_000,cz_000,...,ao_000,bj_000,bi_000,by_000,ck_000,ci_000,cj_000,bt_000,aa_000,class
0,223120.0,210550.0,24.0,0.0,2.0,20.0,22.0,8894.0,0.0,56.0,...,194688.0,17962.0,21046.0,398.0,38068.80,31439.04,97958.40,448.53,448.0,0.0
1,149160.0,151280.0,46.0,0.0,2.0,8.0,128.0,44599.0,0.0,205.0,...,25927250.0,2254536.0,1626396.0,236927.0,3757785.60,27214602.24,615681.60,306241.15,306242.0,NaN
2,223120.0,262700.0,152.0,0.0,0.0,30.0,190.0,91508.0,0.0,6496.0,...,1323854.0,268590.0,98804.0,3400.0,575400.00,1035457.92,45625.92,11485.04,11486.0,NaN
3,223120.0,210550.0,18.0,0.0,0.0,0.0,18.0,276.0,0.0,18.0,...,83204.0,12238.0,41222.0,201.0,52356.48,33396.48,0.00,450.48,450.0,NaN
4,481020.0,407440.0,46.0,0.0,2.0,8.0,128.0,44599.0,0.0,205.0,...,23113102.0,8432244.0,3234330.0,103896.0,15401701.44,23994068.16,0.00,175338.11,175338.0,NaN


## Balance data

In [18]:
# Balance train data minority class
oversample = SMOTE(sampling_strategy='minority', random_state=RANDOM_SEED)
oversample.fit(X_train_filled_nan, y_train)

# # Save the oversample to a file
# with open('train_artifacts\\oversample.pkl', 'wb') as file:
#     pickle.dump(oversample, file)

# # Load the oversample from the file
# with open('train_artifacts\\oversample.pkl', 'rb') as file:
#     oversample = pickle.load(file)

# Data balanced
X_train_balanced, y_train_balanced = oversample.fit_resample(X_train_filled_nan, y_train)
# y train form dataframe to series
y_train_balanced = y_train_balanced['class']

In [19]:
df_train_balanced = X_train_balanced.join(y_train_balanced)
df_train_balanced.head()

,bl_000,bk_000,cg_000,ch_000,cf_000,co_000,ad_000,cx_000,cy_000,cz_000,...,ao_000,bj_000,bi_000,by_000,ck_000,ci_000,cj_000,bt_000,aa_000,class
0,223120.0,210550.0,18.0,0.0,0.0,2.0,72.0,28824.0,0.0,42.0,...,93134.0,11718.0,22412.0,332.0,24934.08,85346.88,0.0,1560.87,1560.0,0
1,182000.0,147000.0,46.0,0.0,2.0,8.0,128.0,44599.0,0.0,205.0,...,1989590.0,184192.0,166962.0,13168.0,237656.64,2263753.92,0.0,39676.28,39676.0,0
2,269700.0,321880.0,304.0,0.0,0.0,484.0,1738.0,95572.0,0.0,36342.0,...,9533838.0,924640.0,2146080.0,74086.0,1666955.52,9946325.76,0.0,164987.86,164988.0,0
3,234600.0,231060.0,46.0,0.0,2.0,8.0,128.0,956014.0,4.0,868.0,...,22910020.0,2113040.0,3986672.0,65953.0,9266841.60,16313824.32,0.0,241465.38,241466.0,0
4,215980.0,185540.0,70.0,0.0,2.0,392.0,152.0,6830.0,0.0,1592.0,...,1900564.0,181094.0,263756.0,18238.0,251109.12,2244048.00,0.0,41802.60,41802.0,0


In [20]:
# Calculate train events proportion
size_train_balanced      = df_train_balanced.shape[0]
non_event_train_balanced = df_train_balanced[df_train_balanced['class'] == 0].shape[0]
event_Train_balanced     = df_train_balanced[df_train_balanced['class'] == 1].shape[0]

non_event_train_balanced_percent = round(non_event_train_balanced / size_train_balanced * 100, 2)
event_train_balanced_percent     = round(event_Train_balanced / size_train_balanced * 100, 2)

non_event_train_balanced_percent, event_train_balanced_percent

(50.0, 50.0)

In [21]:
# Calculate validation events proportion
size_val      = df_val.shape[0]
non_event_val = df_val[df_val['class'] == 0].shape[0]
event_val     = df_val[df_val['class'] == 1].shape[0]

non_event_val_percent = round(non_event_val / size_val * 100, 2)
event_val_percent     = round(event_val / size_val * 100, 2)

non_event_val_percent, event_val_percent

(29.58, 0.56)

## Correlations

In [22]:
# df_train_corr = df_train_balanced.corr().round(2)

# fig = go.Figure()
# fig.add_trace(
#     go.Heatmap(
#         x=df_train_corr.columns,
#         y=df_train_corr.index,
#         z=np.array(df_train_corr),
#         text=df_train_corr.values,
#         texttemplate='%{text:.4f}',
#         colorscale='viridis'))

# fig.update_layout(
#     height=2000,
#     width=2000,
#     yaxis=dict(autorange='reversed'))

# html_file ='temp_plot.html'
# pio.write_html(fig, file=html_file, auto_open=True)

Can you present to me the main factors that point to a possible failure in this system?

# Functions

In [23]:
def calculate_metrics_standard_cutoff(y_true, y_predict):
    no_defect_cost  = 10
    preventive_cost = 25
    corrective_cost = 500
    
    accuracy       = accuracy_score(y_true, y_predict)
    precision      = precision_score(y_true, y_predict, zero_division=0)
    recall         = recall_score(y_true, y_predict, zero_division=0)
    f1score        = f1_score(y_true, y_predict, zero_division=0)
    auc_value      = roc_auc_score(y_true, y_predict)
    tn, fp, fn, tp = confusion_matrix(y_true, y_predict).ravel()
    specificity    = tn / (tn + fp)

    false_positive_rate       = fp / (fp + tn) * 100
    false_negative_rate       = fn / (fn + tp) * 100
    false_classification_rate = (fp + fn) / y_predict.shape[0] * 100

    no_defect_maintenance_cost  = fp * no_defect_cost
    preventive_maintenance_cost = tp * preventive_cost
    corrective_maintenance_cost = fn * corrective_cost
    total_maintenance_cost      = no_defect_maintenance_cost \
                                     + preventive_maintenance_cost \
                                     + corrective_maintenance_cost
    
    metrics = []
    metrics.append([accuracy,
                    precision,
                    recall,
                    f1score,
                    auc_value,
                    specificity,
                    tn, fp, fn, tp,
                    false_positive_rate,
                    false_negative_rate,
                    false_classification_rate,
                    no_defect_maintenance_cost,
                    preventive_maintenance_cost,
                    corrective_maintenance_cost,
                    total_maintenance_cost]  
                  )
    print(f'Accuracy:        {accuracy:.2f}')
    print(f'Precision:       {precision:.2f}')
    print(f'Recall:          {recall:.2f}')
    print(f'F1-score:        {f1score:.2f}')
    print(f'Specificity:     {specificity:.2f}')
    print(f'AUC:             {auc_value:.2f}\n')
    print(f'TN:              {tn:.2f}')
    print(f'TP:              {tp:.2f}')
    print(f'FN:              {fn:.2f}')
    print(f'FP:              {fp:.2f}\n')
    print(f'FP rate:         {false_positive_rate:.2f}%')
    print(f'FN rate:         {false_negative_rate:.2f}%')
    print(f'False clf rate:  {false_classification_rate:.2f}%\n')
    print(f'No defect cost:  {no_defect_maintenance_cost:.2f}')
    print(f'Predictive cost: {preventive_maintenance_cost:.2f}')
    print(f'Corrective cost: {corrective_maintenance_cost:.2f}')
    print(f'Total cost:      {total_maintenance_cost:.2f}\n')

    cm = confusion_matrix(y_true, y_predict)
    ConfusionMatrixDisplay(confusion_matrix=cm).plot()
    plt.show()

    RocCurveDisplay.from_predictions(y_true, y_predict).plot

    return metrics

In [24]:
def calculate_metrics_range_cutoff(dataframe_predict_proba, y_true):
    no_defect_cost  = 10
    preventive_cost = 25
    corrective_cost = 500
    results = []
    for cutoff in np.arange(0.05, 0.99, 0.05):
        y_predict      = np.where(dataframe_predict_proba['proba_event'] > cutoff, 1, 0)
        accuracy       = accuracy_score(y_true, y_predict)
        precision      = precision_score(y_true, y_predict, zero_division=0)
        recall         = recall_score(y_true, y_predict, zero_division=0)
        f1score        = f1_score(y_true, y_predict, zero_division=0)
        auc_value      = roc_auc_score(y_true, y_predict)
        tn, fp, fn, tp = confusion_matrix(y_true, y_predict).ravel()
        specificity    = tn / (tn + fp)

        false_positive_rate       = fp / (fp + tn) * 100
        false_negative_rate       = fn / (fn + tp) * 100
        false_classification_rate = (fp + fn) / y_predict.shape[0] * 100

        no_defect_maintenance_cost  = fp * no_defect_cost
        preventive_maintenance_cost = tp * preventive_cost
        corrective_maintenance_cost = fn * corrective_cost
        total_maintenance_cost      = no_defect_maintenance_cost \
                                       + preventive_maintenance_cost \
                                        + corrective_maintenance_cost
                                
        results.append([cutoff, accuracy, precision,
                        recall, f1score,
                        auc_value, tn, fp, fn, tp,
                        specificity, false_positive_rate,
                        false_negative_rate, false_classification_rate,
                        no_defect_maintenance_cost,
                        preventive_maintenance_cost,
                        corrective_maintenance_cost,
                        total_maintenance_cost]
                    )
        
        df_metrics = pd.DataFrame(results,
                                  columns=['cutoff', 'accuracy', 'precision',
                                           'recall', 'f1score' , 'auc_value',
                                           'tn', 'fp', 'fn', 'tp',
                                           'specificity', 'false_positive_rate',
                                           'false_negative_rate', 'false_classification_rate',
                                           'no_defect_maintenance_cost',
                                           'preventive_maintenance_cost',
                                           'corrective_maintenance_cost',
                                           'total_maintenance_cost']
                                 )
        
    return df_metrics

In [25]:
def plot_metrics(dataframe_proba):
    min_total_cost = dataframe_proba['total_maintenance_cost'].min()
    best_cutoff    = dataframe_proba.loc[dataframe_proba['total_maintenance_cost'] \
                                         == min_total_cost]['cutoff'].values[0]
    fig = make_subplots(rows=3, cols=1,
                        vertical_spacing=0.08,
                        horizontal_spacing=0.06,
                        shared_xaxes=False,
                        subplot_titles=['Rates', 'Metrics', 'Costs']
                        )

    fig.add_trace(go.Scatter(
        x=dataframe_proba['cutoff'],
        y=dataframe_proba['false_negative_rate'],
        name='FN rate',
        mode='lines',
        line=dict(color='#eb3040')
        ),
        row=1, col=1
        )

    fig.add_trace(go.Scatter(
        x=dataframe_proba['cutoff'],
        y=dataframe_proba['false_positive_rate'],
        name='FP rate',
        mode='lines',
        line=dict(color='#eb6949')
        ),
        row=1, col=1
        )
        

    fig.add_trace(go.Scatter(
        x=dataframe_proba['cutoff'],
        y=dataframe_proba['false_classification_rate'],
        name='False classification rate',
        mode='lines',
        line=dict(color='#eb8d27')
        ),
        row=1, col=1
        )

    # ***********************************************
    fig.add_trace(go.Scatter(
        x=dataframe_proba['cutoff'],
        y=dataframe_proba['accuracy'],
        name='Accuracy',
        mode='lines',
        line=dict(color='#85ab71')
        ),
        row=2, col=1
        )

    fig.add_trace(go.Scatter(
        x=dataframe_proba['cutoff'],
        y=dataframe_proba['precision'],
        name='Precision',
        mode='lines',
        line=dict(color='#5e8d6f')
        ),
        row=2, col=1
        )

    fig.add_trace(go.Scatter(
        x=dataframe_proba['cutoff'],
        y=dataframe_proba['recall'],
        name='Recall',
        mode='lines',
        line=dict(color='#58b69e')
        ),
        row=2, col=1
        )

    fig.add_trace(go.Scatter(
        x=dataframe_proba['cutoff'],
        y=dataframe_proba['f1score'],
        name='F1-score',
        mode='lines',
        line=dict(color='#c87499')
        ),
        row=2, col=1
        )

    fig.add_trace(go.Scatter(
        x=dataframe_proba['cutoff'],
        y=dataframe_proba['specificity'],
        name='Specificity',
        mode='lines',
        line=dict(color='#cb4d8e')
        ),
        row=2, col=1
        )

    fig.add_trace(go.Scatter(
        x=dataframe_proba['cutoff'],
        y=dataframe_proba['auc_value'],
        name='AUC',
        mode='lines',
        line=dict(color='#988921')
        ),
        row=2, col=1
        )

    # ***********************************************
    fig.add_trace(go.Scatter(
        x=dataframe_proba['cutoff'],
        y=dataframe_proba['no_defect_maintenance_cost'],
        name='No defect maintenance cost',
        mode='lines',
        line=dict(color='green')
        ),
        row=3, col=1
        )
    
    fig.add_trace(go.Scatter(
        x=dataframe_proba['cutoff'],
        y=dataframe_proba['preventive_maintenance_cost'],
        name='Preventive maintenance cost',
        mode='lines',
        line=dict(color='purple')
        ),
        row=3, col=1
        )
    
    fig.add_trace(go.Scatter(
        x=dataframe_proba['cutoff'],
        y=dataframe_proba['corrective_maintenance_cost'],
        name='Corrective maintenance cost',
        mode='lines',
        line=dict(color='red')
        ),
        row=3, col=1
        )
    
    fig.add_trace(go.Scatter(
        x=dataframe_proba['cutoff'],
        y=dataframe_proba['total_maintenance_cost'],
        name='Total maintenance cost',
        mode='lines',
        line=dict(color='blue')
        ),
        row=3, col=1
        )
    
    fig.add_vline(x=best_cutoff,
                  line_dash='dash',
                  label=dict(text=f'Best cutoff {best_cutoff:.2f} \
                             <br>Lower total cost',
                             xanchor='left',
                             yanchor='top')
                  )

    fig.update_layout(height=1200,
                      width=1000,
                      showlegend=True,
                      margin=dict(l=10, r=10, b=10, t=30, pad=0),
                      xaxis1=dict(title='Cutoff', dtick=0.05, range=(0, 1)),
                      xaxis2=dict(title='Cutoff', dtick=0.05, range=(0, 1)),
                      xaxis3=dict(title='Cutoff', dtick=0.05, range=(0, 1)),
                      yaxis1=dict(title='%'),
                      # yaxis2=dict(title='%'),
                      yaxis3=dict(title='US$'),
                      )

    fig.show()

In [26]:
def plot_cross_validation_score(cv_results, score):
    plot_fig, plot_ax = plt.subplots(figsize=(8, 4))
    plot_ax.plot(cv_results[f'train_{score}'],
                 marker='o',
                 linestyle='-',
                 color='blue',
                 label='Train'
                )
    plot_ax.plot(cv_results[f'test_{score}'],
                 marker='o',
                 linestyle='-',
                 color='red',
                 label='Validation'
                )
    plot_ax.set_xlabel('Iteration')
    plot_ax.set_xticks(range(1, len(cv_results[f'train_{score}'])))
    plot_ax.set_ylabel('Score')
    plot_ax.set_title(f'Cross-validation {score} score')
    plot_ax.legend()
    plot_ax.grid(True)

In [27]:
def calculate_feature_importance(model):
    # Calculate feature importance (Gini)
    feat_importance    = model.feature_importances_
    df_feat_importance = pd.DataFrame({'feature'   : model.feature_names_in_,
                                       'importance': feat_importance}
                                      ).sort_values('importance', ascending=True)

    plt.figure(figsize=(8, 4))
    plt.boxplot(x=df_feat_importance['importance'], vert=False)
    plt.xlabel('Importance')
    plt.title('Feature importance distribution')

    df_feat_importance_no_zero = df_feat_importance[df_feat_importance['importance'] > 0]
    plt.figure(figsize=(8, 10))
    plt.barh(df_feat_importance_no_zero['feature'], df_feat_importance_no_zero['importance'])
    plt.xlabel('Importance')
    plt.ylabel('Features')
    plt.title(f'{df_feat_importance_no_zero.shape[0]} importance greater than 0')

    return df_feat_importance_no_zero

# Model

## Random forest

In [28]:
parameters_rf_clf = {
    "model_parameters": {
        "n_estimators": [5, 100],
        "criterion": ["gini"],
        "max_depth": [2, 13],
        "min_samples_split": [0.2, 1],
        "min_samples_leaf": [1, 7]
    },
    "cross_validation": {
        "n_iterations": 5,
        "folders": 5,
        "scores": [
            "recall",
            "f1_macro",
            "roc_auc",
            "precision_macro"
        ]
    },
    "maintenance_cost": {
        "no_defect_cost": 10,
        "preventive_cost": 25,
        "corrective_cost": 500
    },
    "target": "class",
    "cutoff": 0.3,
    "model_name": "rand_forest",
    "predict_method": "predict_proba"
}

In [29]:
def create_hyper_parameters_rf_clf(parameters: dict):
    range_n_estimators = np.arange(start=round(parameters.get('model_parameters')['n_estimators'][0], 2),
                                   stop=round(parameters.get('model_parameters')['n_estimators'][-1], 2),
                                   step=5
                                   )
    range_max_depth = np.arange(start=round(parameters.get('model_parameters')['max_depth'][0], 2),
                                stop=round(parameters.get('model_parameters')['max_depth'][-1], 2),
                                step=2
                                )
    range_min_samples_split = np.arange(start=round(parameters.get('model_parameters')['min_samples_split'][0], 2),
                                        stop=round(parameters.get('model_parameters')['min_samples_split'][-1], 2),
                                        step=0.1
                                        )
    range_min_samples_leaf = np.arange(start=round(parameters.get('model_parameters')['min_samples_leaf'][0], 2),
                                       stop=round(parameters.get('model_parameters')['min_samples_leaf'][-1], 2),
                                       step=1
                                       )
    param_distributions = {'n_estimators'     : range_n_estimators,
                           'max_depth'        : range_max_depth,
                           'min_samples_split': range_min_samples_split,
                           'min_samples_leaf' : range_min_samples_leaf
                          }
    
    return param_distributions

### Simple model

In [30]:
rf_clf = RandomForestClassifier(random_state=RANDOM_SEED)
rf_clf.fit(X_train_balanced, y_train_balanced)

RandomForestClassifier()

#### Cross-validation scores

In [31]:
cv_results_rf_clf = cross_validate(estimator=rf_clf,
                                   X=X_train_balanced,
                                   y=y_train_balanced,
                                   scoring=parameters_rf_clf.get('cross_validation')['scores'],
                                   cv=parameters_rf_clf.get('cross_validation')['folders'],
                                   verbose=0,
                                   return_train_score=True,
                                   error_score=np.nan
                                   )

# Calculate cross-validation scores
for score in parameters_rf_clf.get('cross_validation')['scores']:
    cv_train_mean = cv_results_rf_clf[f'train_{score}'].mean().round(4)
    cv_test_mean  = cv_results_rf_clf[f'test_{score}'].mean().round(4)
    plot_cross_validation_score(cv_results_rf_clf, score)
    print(f'>>>>>>>>> CV {score} train mean: {cv_train_mean}')
    print(f'>>>>>>>>> CV {score} test mean:  {cv_test_mean}')

KeyboardInterrupt: 

In [ ]:
y_pred_rf_clf       = rf_clf.predict(X_val_filled_nan)
y_pred_proba_rf_clf = rf_clf.predict_proba(X_val_filled_nan)

In [ ]:
metrics_rf_clf = calculate_metrics_standard_cutoff(y_val, y_pred_rf_clf)

In [ ]:
df_pred_proba_rf_clf = pd.DataFrame(y_pred_proba_rf_clf,
                                    columns=['proba_non_event', 'proba_event']
                                    )
df_pred_proba_rf_clf

In [ ]:
df_metrics_rf_clf = calculate_metrics_range_cutoff(df_pred_proba_rf_clf, y_val)
df_metrics_rf_clf

In [ ]:
plot_metrics(df_metrics_rf_clf)

### Tuning

In [32]:
param_dist_rf_clf = create_hyper_parameters_rf_clf(parameters_rf_clf)
param_dist_rf_clf

{'n_estimators': array([ 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85,
        90, 95]),
 'max_depth': array([ 2,  4,  6,  8, 10, 12]),
 'min_samples_split': array([0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]),
 'min_samples_leaf': array([1, 2, 3, 4, 5, 6])}

In [33]:
random_search_rf_clf = RandomizedSearchCV(estimator=rf_clf,
                                          param_distributions=param_dist_rf_clf,
                                          verbose=3,
                                          cv=parameters_rf_clf.get('cross_validation')['folders'],
                                          n_iter=parameters_rf_clf.get('cross_validation')['n_iterations'],
                                          random_state=RANDOM_SEED,
                                          scoring=make_scorer(f1_score)
                                          )
random_search_rf_clf.fit(X_train_balanced, y_train_balanced)

best_params_rf_clf = random_search_rf_clf.best_params_
rf_clf_tuning = RandomForestClassifier(**best_params_rf_clf, random_state=RANDOM_SEED)
rf_clf_tuning.fit(X_train_balanced, y_train_balanced)
print(f'\nBest parameters:\n{best_params_rf_clf}\n')

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV 1/5] END max_depth=4, min_samples_leaf=4, min_samples_split=0.30000000000000004, n_estimators=15;, score=0.952 total time=   2.0s
[CV 2/5] END max_depth=4, min_samples_leaf=4, min_samples_split=0.30000000000000004, n_estimators=15;, score=0.949 total time=   2.0s
[CV 3/5] END max_depth=4, min_samples_leaf=4, min_samples_split=0.30000000000000004, n_estimators=15;, score=0.955 total time=   2.0s
[CV 4/5] END max_depth=4, min_samples_leaf=4, min_samples_split=0.30000000000000004, n_estimators=15;, score=0.955 total time=   1.8s
[CV 5/5] END max_depth=4, min_samples_leaf=4, min_samples_split=0.30000000000000004, n_estimators=15;, score=0.949 total time=   1.8s
[CV 1/5] END max_depth=6, min_samples_leaf=6, min_samples_split=0.7000000000000002, n_estimators=10;, score=0.000 total time=   0.1s
[CV 2/5] END max_depth=6, min_samples_leaf=6, min_samples_split=0.7000000000000002, n_estimators=10;, score=0.000 total time=   0.1s
[CV 

#### Cross-validation scores

In [ ]:
cv_results_rf_clf_tuning = cross_validate(estimator=rf_clf_tuning,
                                          X=X_train_balanced,
                                          y=y_train_balanced,
                                          scoring=parameters_rf_clf.get('cross_validation')['scores'],
                                          cv=parameters_rf_clf.get('cross_validation')['folders'],
                                          verbose=0,
                                          return_train_score=True,
                                          error_score=np.nan
                                          )

# Calculate cross-validation scores
for score in parameters_rf_clf.get('cross_validation')['scores']:
    cv_train_mean = cv_results_rf_clf_tuning[f'train_{score}'].mean().round(4)
    cv_test_mean  = cv_results_rf_clf_tuning[f'test_{score}'].mean().round(4)
    plot_cross_validation_score(cv_results_rf_clf_tuning, score)
    print(f'>>>>>>>>> CV {score} train mean: {cv_train_mean}')
    print(f'>>>>>>>>> CV {score} test mean:  {cv_test_mean}')

In [ ]:
y_pred_rf_clf_tuning       = rf_clf_tuning.predict(X_val_filled_nan)
y_pred_proba_rf_clf_tuning = rf_clf_tuning.predict_proba(X_val_filled_nan)

In [ ]:
metrics_rf_clf_tuning = calculate_metrics_standard_cutoff(y_val, y_pred_rf_clf_tuning)

In [ ]:
df_pred_proba_rf_clf_tuning = pd.DataFrame(y_pred_proba_rf_clf_tuning,
                                           columns=['proba_non_event', 'proba_event']
                                           )
df_metrics_rf_clf_tuning = calculate_metrics_range_cutoff(df_pred_proba_rf_clf_tuning, y_val)
df_metrics_rf_clf_tuning
plot_metrics(df_metrics_rf_clf_tuning)

### Feature importance

In [ ]:
df_feat_importance_rf_clf = calculate_feature_importance(rf_clf_tuning)
df_feat_importance_rf_clf.head()

In [ ]:
important_features_rf_clf         = df_feat_importance_rf_clf['feature'].to_list()
X_train_important_features_rf_clf = X_train_balanced[important_features_rf_clf]
X_val_important_features_rf_clf   = X_val_filled_nan[important_features_rf_clf]

rf_clf_tuning.fit(X_train_important_features_rf_clf, y_train_balanced)

#### Cross-validation scores

In [ ]:
cv_results_rf_clf_important_features = cross_validate(estimator=rf_clf_tuning,
                                                      X=X_train_important_features_rf_clf,
                                                      y=y_train_balanced,
                                                      scoring=parameters_rf_clf.get('cross_validation')['scores'],
                                                      cv=parameters_rf_clf.get('cross_validation')['folders'],
                                                      verbose=0,
                                                      return_train_score=True,
                                                      error_score=np.nan
                                                      )

# Calculate cross-validation scores
for score in parameters_rf_clf.get('cross_validation')['scores']:
    cv_train_mean = cv_results_rf_clf_important_features[f'train_{score}'].mean().round(4)
    cv_test_mean  = cv_results_rf_clf_important_features[f'test_{score}'].mean().round(4)
    plot_cross_validation_score(cv_results_rf_clf_important_features, score)
    print(f'>>>>>>>>> CV {score} train mean: {cv_train_mean}')
    print(f'>>>>>>>>> CV {score} test mean:  {cv_test_mean}')

In [ ]:
y_pred_rf_clf_important_features       = rf_clf_tuning.predict(X_val_important_features_rf_clf)
y_pred_proba_rf_clf_important_features = rf_clf_tuning.predict_proba(X_val_important_features_rf_clf)

In [ ]:
metrics_rf_clf_important_features = calculate_metrics_standard_cutoff(y_val, y_pred_rf_clf_important_features)

In [ ]:
df_pred_proba_rf_clf_important_features = pd.DataFrame(y_pred_proba_rf_clf_important_features,
                                                       columns=['proba_non_event', 'proba_event']
                                                       )
df_metrics_rf_clf_important_features = calculate_metrics_range_cutoff(df_pred_proba_rf_clf_important_features, y_val)
df_metrics_rf_clf_important_features
plot_metrics(df_metrics_rf_clf_important_features)

## XGBoost

In [ ]:
parameters_xgb_clf = {
    "model_parameters": {
        "n_estimators": [60, 80],
        "max_depth": [10, 13],
        "learning_rate": [0.05, 0.1],
        "subsample": [0.8, 1],
        "colsample_bytree": [0.6, 1]
    },
    "cross_validation": {
        "n_iterations": 10,
        "folders": 5,
        "scores": [
            "accuracy",
            "f1_macro",
            "roc_auc",
            "precision_macro"
        ]
    },
    "maintenance_cost": {
        "no_defect_cost": 10,
        "preventive_cost": 25,
        "corrective_cost": 500
    },
    "target": "class",
    "cutoff": 0.3,
    "model_name": "xgb_clf",
    "predict_method": "predict_proba"
}

In [ ]:
def create_hyper_parameters_range_xgb(parameters: dict):
    range_n_estimators = np.arange(start=round(parameters.get('model_parameters')['n_estimators'][0], 2),
                                   stop=round(parameters.get('model_parameters')['n_estimators'][-1], 2),
                                   step=2
                                   )
    range_max_depth = np.arange(start=round(parameters.get('model_parameters')['max_depth'][0], 2),
                                stop=round(parameters.get('model_parameters')['max_depth'][-1], 2),
                                step=1
                                )
    range_learning_rate = np.arange(start=round(parameters.get('model_parameters')['learning_rate'][0], 2),
                                    stop=round(parameters.get('model_parameters')['learning_rate'][-1], 2),
                                    step=0.01
                                    )
    range_subsample = np.arange(start=round(parameters.get('model_parameters')['subsample'][0], 2),
                                stop=round(parameters.get('model_parameters')['subsample'][-1], 2),
                                step=0.015
                                )
    range_colsample_bytree = np.arange(start=round(parameters.get('model_parameters')['colsample_bytree'][0], 2),
                                       stop=round(parameters.get('model_parameters')['colsample_bytree'][-1], 2),
                                       step=0.05
                                       )
    param_distributions = {'n_estimators'     : range_n_estimators,
                           'max_depth'        : range_max_depth,
                           'learning_rate'    : range_learning_rate,
                           'subsample'        : range_subsample,
                           'colsample_bytree' :range_colsample_bytree
                          }
    
    return param_distributions

### Simple model

In [ ]:
xgb_clf = XGBClassifier(random_state=RANDOM_SEED, verbosity=0)
xgb_clf.fit(X_train_balanced, y_train_balanced)

#### Cross-validation scores

In [ ]:
cv_results_xgb_clf = cross_validate(estimator=xgb_clf,
                                    X=X_train_balanced,
                                    y=y_train_balanced,
                                    scoring=parameters_xgb_clf.get('cross_validation')['scores'],
                                    cv=parameters_xgb_clf.get('cross_validation')['folders'],
                                    verbose=0,
                                    return_train_score=True,
                                    error_score=np.nan
                                    )

# Calculate cross-validation scores
for score in parameters_xgb_clf.get('cross_validation')['scores']:
    cv_train_mean = cv_results_xgb_clf[f'train_{score}'].mean().round(4)
    cv_test_mean  = cv_results_xgb_clf[f'test_{score}'].mean().round(4)
    plot_cross_validation_score(cv_results_xgb_clf, score)
    print(f'>>>>>>>>> CV {score} train mean: {cv_train_mean}')
    print(f'>>>>>>>>> CV {score} test mean:  {cv_test_mean}')

In [ ]:
y_pred_xgb_clf       = xgb_clf.predict(X_val_filled_nan)
y_pred_proba_xgb_clf = xgb_clf.predict_proba(X_val_filled_nan)

In [ ]:
metrics_xgb_clf = calculate_metrics_standard_cutoff(y_val, y_pred_xgb_clf)

In [ ]:
df_pred_proba_xgb_clf = pd.DataFrame(y_pred_proba_xgb_clf,
                                     columns=['proba_non_event', 'proba_event']
                                     )
df_metrics_xgb_clf = calculate_metrics_range_cutoff(df_pred_proba_xgb_clf, y_val)
plot_metrics(df_metrics_xgb_clf)

### Tuning

In [ ]:
param_dist_xgb_clf = create_hyper_parameters_range_xgb(parameters_xgb_clf)
param_dist_xgb_clf

In [ ]:
random_search_xgb_clf = RandomizedSearchCV(estimator=xgb_clf,
                                           param_distributions=param_dist_xgb_clf,
                                           verbose=3,
                                           cv=parameters_xgb_clf.get('cross_validation')['folders'],
                                           n_iter=parameters_xgb_clf.get('cross_validation')['n_iterations'],
                                           random_state=RANDOM_SEED,
                                           scoring=make_scorer(f1_score)
                                           )
random_search_xgb_clf.fit(X_train_balanced, y_train_balanced)

best_params_xgb_clf = random_search_xgb_clf.best_params_
xgb_clf_tuning = XGBClassifier(**best_params_xgb_clf, random_state=RANDOM_SEED)
xgb_clf_tuning.fit(X_train_balanced, y_train_balanced)
print(f'\nBest parameters:\n{best_params_xgb_clf}')

#### Cross-validation scores

In [ ]:
cv_results_xgb_clf_tuning = cross_validate(estimator=xgb_clf_tuning,
                                           X=X_train_balanced,
                                           y=y_train_balanced,
                                           scoring=parameters_xgb_clf.get('cross_validation')['scores'],
                                           cv=parameters_xgb_clf.get('cross_validation')['folders'],
                                           verbose=0,
                                           return_train_score=True,
                                           error_score=np.nan
                                           )

# Calculate cross-validation scores
for score in parameters_xgb_clf.get('cross_validation')['scores']:
    cv_train_mean = cv_results_xgb_clf_tuning[f'train_{score}'].mean().round(4)
    cv_test_mean  = cv_results_xgb_clf_tuning[f'test_{score}'].mean().round(4)
    plot_cross_validation_score(cv_results_xgb_clf_tuning, score)
    print(f'>>>>>>>>> CV {score} train mean: {cv_train_mean}')
    print(f'>>>>>>>>> CV {score} test mean:  {cv_test_mean}')

In [ ]:
y_pred_xgb_clf_tuning       = xgb_clf.predict(X_val_filled_nan)
y_pred_proba_xgb_clf_tuning = xgb_clf.predict_proba(X_val_filled_nan)

In [ ]:
metrics_xgb_clf_tuning = calculate_metrics_standard_cutoff(y_val, y_pred_xgb_clf_tuning)

In [ ]:
df_pred_proba_xgb_clf_tuning = pd.DataFrame(y_pred_proba_xgb_clf_tuning,
                                            columns=['proba_non_event', 'proba_event']
                                           )
df_metrics_xgb_clf_tuning = calculate_metrics_range_cutoff(df_pred_proba_xgb_clf_tuning, y_val)
df_metrics_xgb_clf_tuning
plot_metrics(df_metrics_xgb_clf_tuning)

### Feature importance

In [ ]:
df_feat_importance_xgb_clf = calculate_feature_importance(xgb_clf)
df_feat_importance_xgb_clf.head()

In [ ]:
important_features_xgb_clf         = df_feat_importance_xgb_clf['feature'].to_list()
X_train_important_features_xgb_clf = X_train_balanced[important_features_xgb_clf]
X_val_important_features_xgb_clf   = X_val_filled_nan[important_features_xgb_clf]

xgb_clf_tuning.fit(X_train_important_features_xgb_clf, y_train_balanced)

#### Cross-validation scores

In [ ]:
cv_results_xgb_clf_important_features = cross_validate(estimator=xgb_clf_tuning,
                                                       X=X_train_important_features_xgb_clf,
                                                       y=y_train_balanced,
                                                       scoring=parameters_xgb_clf.get('cross_validation')['scores'],
                                                       cv=parameters_xgb_clf.get('cross_validation')['folders'],
                                                       verbose=0,
                                                       return_train_score=True,
                                                       error_score=np.nan
                                                       )

# Calculate cross-validation scores
for score in parameters_xgb_clf.get('cross_validation')['scores']:
    cv_train_mean = cv_results_xgb_clf_important_features[f'train_{score}'].mean().round(4)
    cv_test_mean  = cv_results_xgb_clf_important_features[f'test_{score}'].mean().round(4)
    plot_cross_validation_score(cv_results_xgb_clf_important_features, score)
    print(f'>>>>>>>>> CV {score} train mean: {cv_train_mean}')
    print(f'>>>>>>>>> CV {score} test mean:  {cv_test_mean}')

In [ ]:
y_pred_xgb_clf_important_features       = xgb_clf_tuning.predict(X_val_important_features_xgb_clf)
y_pred_proba_xgb_clf_important_features = xgb_clf_tuning.predict_proba(X_val_important_features_xgb_clf)

In [ ]:
metrics_xgb_clf_important_features = calculate_metrics_standard_cutoff(y_val, y_pred_xgb_clf_important_features)

In [ ]:
df_pred_proba_xgb_clf_important_features = pd.DataFrame(y_pred_proba_xgb_clf_important_features,
                                                        columns=['proba_non_event', 'proba_event']
                                                       )
df_metrics_xgb_clf_important_features = calculate_metrics_range_cutoff(df_pred_proba_xgb_clf_important_features, y_val)
df_metrics_xgb_clf_important_features
plot_metrics(df_metrics_xgb_clf_important_features)

## MLPClassifier

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train_balanced)

X_train_scaled = scaler.transform(X_train_balanced)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train_balanced.columns)
df_train_scaled = df_target.join(X_train_scaled)
df_train_scaled.rename(columns={'class': 'target'}, inplace=True)

X_val_scaled = scaler.transform(X_val_filled_nan)
X_val_scaled = pd.DataFrame(X_val_scaled, columns=X_val_filled_nan.columns)

In [ ]:
parameters_mlp_clf = {'hyper_parameters': {'hidden_layer_sizes': [(200), (100, 50), (200, 100, 50)],
                                           'max_iter': [500, 1000],
                                           'activation': ['relu'],
                                           'solver': ['adam'],
                                           'alpha': [0.0001, 0.001],
                                           'learning_rate': ['adaptive']
                                          },
                      "cross_validation": {"n_iterations": 5,
                                           "folders": 5,
                                           "scores": [
                                               "recall",
                                               "f1_macro",
                                               "roc_auc",
                                               "precision_macro"]
                                          }
                      }

### Simple model

In [ ]:
mlp_clf = MLPClassifier()
mlp_clf.fit(X_train_scaled, y_train_balanced)

#### Cross-validation scores

In [ ]:
cv_results_mlp_clf = cross_validate(estimator=mlp_clf,
                                    X=X_train_scaled,
                                    y=y_train_balanced,
                                    scoring=parameters_mlp_clf.get('cross_validation')['scores'],
                                    cv=parameters_mlp_clf.get('cross_validation')['folders'],
                                    verbose=0,
                                    return_train_score=True,
                                    error_score=np.nan
                                    )

# Calculate cross-validation scores
for score in parameters_mlp_clf.get('cross_validation')['scores']:
    cv_train_mean = cv_results_mlp_clf[f'train_{score}'].mean().round(4)
    cv_test_mean  = cv_results_mlp_clf[f'test_{score}'].mean().round(4)
    plot_cross_validation_score(cv_results_mlp_clf, score)
    print(f'>>>>>>>>> CV {score} train mean: {cv_train_mean}')
    print(f'>>>>>>>>> CV {score} test mean:  {cv_test_mean}')

In [ ]:
plt.plot(mlp_clf.loss_curve_)
plt.title("Loss Curve", fontsize=14)
plt.xlabel('Iterations')
plt.ylabel('Cost')
plt.show()

In [ ]:
y_pred_mlp_clf       = mlp_clf.predict(X_val_scaled)
y_pred_proba_mlp_clf = mlp_clf.predict_proba(X_val_scaled)

In [ ]:
metrics_mlp_clf = calculate_metrics_standard_cutoff(y_val, y_pred_mlp_clf)

In [ ]:
df_pred_proba_mlp_clf = pd.DataFrame(y_pred_proba_mlp_clf,
                                     columns=['proba_non_event', 'proba_event']
                                     )
df_metrics_mlp_clf = calculate_metrics_range_cutoff(df_pred_proba_mlp_clf, y_val)
plot_metrics(df_metrics_mlp_clf)

## AdaBoosting

### Random forest

In [ ]:
ada_rf_clf = AdaBoostClassifier(estimator=RandomForestClassifier(verbose=0, random_state=RANDOM_SEED),
                                random_state=RANDOM_SEED,
                                algorithm='SAMME'
                               )
ada_rf_clf.fit(X_train_scaled, y_train_balanced)

#### Cross-validation scores

In [ ]:
cv_results_ada_rf_clf = cross_validate(estimator=ada_rf_clf,
                                       X=X_train_scaled,
                                       y=y_train_balanced,
                                       scoring=parameters_rf_clf.get('cross_validation')['scores'],
                                       cv=parameters_rf_clf.get('cross_validation')['folders'],
                                       verbose=0,
                                       return_train_score=True,
                                       error_score=np.nan
                                       )

# Calculate cross-validation scores
for score in parameters_rf_clf.get('cross_validation')['scores']:
    cv_train_mean = cv_results_ada_rf_clf[f'train_{score}'].mean().round(4)
    cv_test_mean  = cv_results_ada_rf_clf[f'test_{score}'].mean().round(4)
    plot_cross_validation_score(cv_results_ada_rf_clf, score)
    print(f'>>>>>>>>> CV {score} train mean: {cv_train_mean}')
    print(f'>>>>>>>>> CV {score} test mean:  {cv_test_mean}')

In [ ]:
y_pred_ada_rf_clf       = ada_rf_clf.predict(X_val_scaled)
y_pred_proba_ada_rf_clf = ada_rf_clf.predict_proba(X_val_scaled)

In [ ]:
metrics_ada_rf_clf = calculate_metrics_standard_cutoff(y_val, y_pred_ada_rf_clf)

In [ ]:
df_pred_proba_ada_rf_clf = pd.DataFrame(y_pred_proba_ada_rf_clf,
                                        columns=['proba_non_event', 'proba_event']
                                       )
df_metrics_ada_rf_clf = calculate_metrics_range_cutoff(df_pred_proba_ada_rf_clf, y_val)
plot_metrics(df_metrics_ada_rf_clf)

### Decision tree

In [ ]:
from    sklearn.tree import DecisionTreeClassifier

In [ ]:
ada_dt_clf = AdaBoostClassifier(estimator=DecisionTreeClassifier(random_state=RANDOM_SEED),
                                random_state=RANDOM_SEED,
                                algorithm='SAMME'
                                )
ada_dt_clf.fit(X_train_scaled, y_train_balanced)

#### Cross-validation scores

In [ ]:
cv_results_ada_dt_clf = cross_validate(estimator=ada_dt_clf,
                                       X=X_train_scaled,
                                       y=y_train_balanced,
                                       scoring=parameters_rf_clf.get('cross_validation')['scores'],
                                       cv=parameters_rf_clf.get('cross_validation')['folders'],
                                       verbose=0,
                                       return_train_score=True,
                                       error_score=np.nan
                                       )

# Calculate cross-validation scores
for score in parameters_rf_clf.get('cross_validation')['scores']:
    cv_train_mean = cv_results_ada_dt_clf[f'train_{score}'].mean().round(4)
    cv_test_mean  = cv_results_ada_dt_clf[f'test_{score}'].mean().round(4)
    plot_cross_validation_score(cv_results_ada_dt_clf, score)
    print(f'>>>>>>>>> CV {score} train mean: {cv_train_mean}')
    print(f'>>>>>>>>> CV {score} test mean:  {cv_test_mean}')

In [ ]:
y_pred_ada_dt_clf       = ada_dt_clf.predict(X_val_scaled)
y_pred_proba_ada_dt_clf = ada_dt_clf.predict_proba(X_val_scaled)

In [ ]:
metrics_ada_dt_clf = calculate_metrics_standard_cutoff(y_val, y_pred_ada_dt_clf)

In [ ]:
df_pred_proba_ada_dt_clf = pd.DataFrame(y_pred_proba_ada_dt_clf,
                                        columns=['proba_non_event', 'proba_event']
                                        )
df_metrics_ada_dt_clf = calculate_metrics_range_cutoff(df_pred_proba_ada_dt_clf, y_val)
plot_metrics(df_metrics_ada_dt_clf)

# Compare models

## Standard cutoff

In [ ]:
columns = ['accuracy',
           'precision',
           'recall',
           'f1score',
           'auc_value',
           'specificity',
           'tn', 'fp', 'fn', 'tp',
           'false_positive_rate',
           'false_negative_rate',
           'false_classification_rate',
           'no_defect_maintenance_cost',
           'preventive_maintenance_cost',
           'corrective_maintenance_cost',
           'total_maintenance_cost']

df_metrics_standard_cutoff = pd.DataFrame(metrics_rf_clf,
                                          columns=columns
                                         )
df_metrics_standard_cutoff = pd.concat([df_metrics_standard_cutoff,
                                         pd.DataFrame(metrics_rf_clf_tuning, columns=columns)], axis=0)
df_metrics_standard_cutoff = pd.concat([df_metrics_standard_cutoff,
                                         pd.DataFrame(metrics_rf_clf_important_features, columns=columns)], axis=0)
df_metrics_standard_cutoff = pd.concat([df_metrics_standard_cutoff,
                                         pd.DataFrame(metrics_xgb_clf, columns=columns)], axis=0)
df_metrics_standard_cutoff = pd.concat([df_metrics_standard_cutoff,
                                         pd.DataFrame(metrics_xgb_clf_tuning, columns=columns)], axis=0)
df_metrics_standard_cutoff = pd.concat([df_metrics_standard_cutoff,
                                         pd.DataFrame(metrics_xgb_clf_important_features, columns=columns)], axis=0)
df_metrics_standard_cutoff = pd.concat([df_metrics_standard_cutoff,
                                         pd.DataFrame(metrics_mlp_clf, columns=columns)], axis=0)
df_metrics_standard_cutoff = pd.concat([df_metrics_standard_cutoff,
                                         pd.DataFrame(metrics_ada_rf_clf, columns=columns)], axis=0)
df_metrics_standard_cutoff = pd.concat([df_metrics_standard_cutoff,
                                         pd.DataFrame(metrics_ada_dt_clf, columns=columns)], axis=0)
df_metrics_standard_cutoff['model'] = ['rf_clf',
                                       'rf_clf_tuning',
                                       'ada_rf_clf_tuning',
                                       'xgb_clf',
                                       'xgb_clf_tuning',
                                       'xgb_clf_important_features',
                                       'mlp_clf',
                                       'ada_rf_clf',
                                       'ada_dt_clf'
                                       ]
df_metrics_standard_cutoff

In [ ]:
lower_total_cost = df_metrics_standard_cutoff['total_maintenance_cost'].min()
select_model = df_metrics_standard_cutoff[df_metrics_standard_cutoff['total_maintenance_cost'] == lower_total_cost]
select_model

In [ ]:
for column in df_metrics_standard_cutoff.columns:
    if column != 'model':
        fig = px.bar(df_metrics_standard_cutoff,
                x='model',
                y=column,
                text_auto=True,
                text=column
                )
        fig.update_traces(texttemplate='%{text:.3f}')

        fig.update_layout(height=400,
                        width=800,
                        showlegend=True,
                        margin=dict(l=10, r=10, b=10, t=10),
                        yaxis = dict(title=column.title()),
                        xaxis = dict(title='Model')
                        )
        fig.show()

## Various cutoffs

In [ ]:
df_compare_metrics = df_metrics_rf_clf.copy()
df_compare_metrics['model'] = 'rf_clf'

In [ ]:
df_metrics_rf_clf_tuning['model']              = 'rf_clf_tuning'
df_metrics_rf_clf_important_features['model']  = 'rf_clf_important_features'
df_metrics_xgb_clf['model']                    = 'xgb_clf'
df_metrics_xgb_clf_tuning['model']             = 'xgb_clf_tuning'
df_metrics_xgb_clf_important_features['model'] = 'xgb_clf_important_features'
df_metrics_mlp_clf['model']                    = 'mlp_clf'
df_metrics_ada_rf_clf['model']                 = 'ada_rf_clf'
df_metrics_ada_dt_clf['model']                 = 'ada_dt_clf'

df_compare_metrics = pd.concat([df_compare_metrics, df_metrics_rf_clf_tuning], axis=0)
df_compare_metrics = pd.concat([df_compare_metrics, df_metrics_rf_clf_important_features], axis=0)
df_compare_metrics = pd.concat([df_compare_metrics, df_metrics_xgb_clf], axis=0)
df_compare_metrics = pd.concat([df_compare_metrics, df_metrics_xgb_clf_tuning], axis=0)
df_compare_metrics = pd.concat([df_compare_metrics, df_metrics_xgb_clf_important_features], axis=0)
df_compare_metrics = pd.concat([df_compare_metrics, df_metrics_mlp_clf], axis=0)
df_compare_metrics = pd.concat([df_compare_metrics, df_metrics_ada_rf_clf], axis=0)
df_compare_metrics = pd.concat([df_compare_metrics, df_metrics_ada_dt_clf], axis=0)

df_compare_metrics

In [ ]:
df_compare_metrics.groupby(['cutoff', 'model']).mean()

In [ ]:
min_total_cost = df_compare_metrics['total_maintenance_cost'].min()
df_compare_metrics[df_compare_metrics['total_maintenance_cost'] == min_total_cost]

In [ ]:
df_compare_metrics[df_compare_metrics['cutoff'] == 0.5]

# Test

In [ ]:
df_test = pd.read_csv('data\\original\\air_system_present_year.csv',
                      encoding='utf-8',
                      na_values=['na']
                      )

In [ ]:
df_test_target = df_test[['class']]
# Change class to int dummies
map_class = {'neg': 0, 'pos': 1}
df_test_target.loc[:, 'class'] = df_test_target['class'].map(map_class)
y_test = df_test_target['class'].astype(int)
y_test

In [ ]:
# Calculate events proportion
total_test     = df_test_target.shape[0]
non_event_test = df_test_target[df_test_target['class'] == 0].shape[0]
event_test     = df_test_target[df_test_target['class'] == 1].shape[0]

non_event_test_percent = round(non_event_test / total_test * 100, 2)
event_test_percent     = round(event_test / total_test * 100, 2)

non_event_test_percent, event_test_percent

In [ ]:
df_test_features = df_test[X_train_balanced.columns]
df_test_features = df_test_features.astype(float)
df_test_features

In [ ]:
df_test_features.isna().sum().sort_values(ascending=False)

## Input data on NaN

In [ ]:
X_test_filled_nan = imputer.transform(df_test_features)
X_test_filled_nan = pd.DataFrame(X_test_filled_nan, columns=df_test_features.columns)
X_test_filled_nan.isna().sum().sum()

In [ ]:
X_test_filled_nan.head()

# Predict

In [ ]:
select_model

In [ ]:
rf_clf_tuning.fit(X_train_balanced, y_train_balanced)

In [ ]:
y_pred_test = rf_clf_tuning.predict(X_test_filled_nan)
y_pred_proba_test = rf_clf_tuning.predict_proba(X_test_filled_nan)

In [ ]:
metrics_test = calculate_metrics_standard_cutoff(y_test, y_pred_test)

In [ ]:
df_pred_proba_rf_clf_tuning = pd.DataFrame(y_pred_proba_rf_clf_tuning,
                                           columns=['proba_non_event', 'proba_event']
                                           )
df_metrics_rf_clf_tuning = calculate_metrics_range_cutoff(df_pred_proba_rf_clf_tuning, y_val)
df_metrics_rf_clf_tuning
plot_metrics(df_metrics_rf_clf_tuning)

# Historical data

In [ ]:
min_test_cost = df_metrics_rf_clf_tuning['total_maintenance_cost'].min()
min_test_cost

In [ ]:
historical_cost = pd.DataFrame(data=[[24000], [26000], [28999], [32000], [37000], [min_test_cost]],
                               index=[2016, 2017, 2018, 2019, 2020, 2021],
                               columns=['cost']
                               )
historical_cost['pct_change'] = historical_cost['cost'].pct_change(periods=1) * 100
historical_cost['amount_change'] = historical_cost['cost'].diff()

historical_cost

In [ ]:
total_historical_cost = historical_cost.sum().values[0]
total_historical_cost

In [ ]:
fig = make_subplots()

fig.add_trace(go.Bar(
    x=historical_cost.index[:-1],
    y=historical_cost['cost'][:-1],
    name='Historical cost',
    marker=dict(color='red'),
    text=historical_cost['cost'][:-1])
    )

fig.add_trace(go.Bar(
    x=historical_cost.index[-1:],
    y=historical_cost['cost'][-1:],
    name='Estimated cost',
    marker=dict(color='Blue'),
    text=historical_cost['cost'][-1:])
    )

fig.update_layout(height=400,
                  width=800,
                  showlegend=True,
                  margin=dict(l=10, r=10, b=10, t=35, pad=0),
                  xaxis=dict(title=None),
                  yaxis=dict(title='US$', titlefont=dict(size=14)),
                  title=f"US$ {abs(historical_cost['amount_change'][-1:].values[0]):.2f} saved with machine learning"
                  )
fig.show()

In [ ]:
fig = make_subplots()

fig.add_trace(go.Bar(
    x=historical_cost.index[:-1],
    y=historical_cost['pct_change'][:-1],
    name='Historical cost',
    marker=dict(color='red'),
    text=historical_cost['pct_change'][:-1].round(2))
    )

fig.add_trace(go.Bar(
    x=historical_cost.index[-1:],
    y=historical_cost['pct_change'][-1:],
    name='Estimated cost',
    marker=dict(color='blue'),
    text=historical_cost['pct_change'][-1:].round(2))
    )

fig.update_traces(texttemplate='%{text:.2f}%')

fig.update_layout(height=400,
                  width=800,
                  showlegend=True,
                  margin=dict(l=10, r=10, b=10, t=35, pad=0),
                  xaxis=dict(title=None),
                  yaxis=dict(title='%', titlefont=dict(size=14)),
                  title='Percentage cost variation through years'
                  )
fig.show()